In [14]:
%run partialOrder.ipynb

........................................................................................................
----------------------------------------------------------------------
Ran 104 tests in 1.729s

OK
........................................................................................................
----------------------------------------------------------------------
Ran 104 tests in 1.488s

OK
........................................................................................................
----------------------------------------------------------------------
Ran 104 tests in 1.518s

OK


In [15]:
from lark import Lark, Tree, Token

class Interpreter:
    
    def is_valid(self, relation, universe):
        if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
            raise Exception(f"Relation - {relation} not valid for universe - {universe}!")
    
    def __init__(self, variables=dict(), universe=FiniteSet(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, universe)
                
        self._variables = variables
        self._universe = FiniteSet(*universe)
        self._parser = Lark('''
                        RELATION: "P".."Z"
                        SET: "A".."O"
                        ?logical_operator: special_operator
                             | logical_operator "and" special_operator  -> conjunction
                             | logical_operator "or"  special_operator  -> disjunction                   
                             | logical_operator "->"  special_operator  -> implication
                             | logical_operator "=="  special_operator  -> equal
                             | logical_operator "!="  special_operator  -> not_equal
                             | logical_operator ">"   special_operator  -> greater_than
                             | logical_operator "<"   special_operator  -> lower_than
                             | logical_operator ">="  special_operator  -> greater_equal
                             | logical_operator "<="  special_operator  -> lower_equal                   
                        ?special_operator: properties
                             | "not" logical_operator      -> negation
                             | product "is" properties     -> is
                             | product "is not" properties -> is_not
                        ?properties: product    
                             | "total_order"   -> total_order
                             | "transitive"    -> transitive
                             | "lattice"       -> lattice
                             | "reflexive"     -> reflexive
                             | "symmetric"     -> symmetric
                             | "asymmetric"    -> asymmetric
                             | "antisymmetric" -> antisymmetric
                        ?product: atom
                             | product "°" atom   -> composition
                             | product "&" atom   -> intersection
                             | product "|" atom   -> union
                             | product "-" atom   -> difference
                             | product "^" atom   -> symmetric_difference
                             | product "*" atom   -> multiplication
                        ?atom: "("logical_operator")"
                             | SET 
                             | RELATION
                        %import common.WS
                        %ignore WS
                 ''', start='logical_operator')
        
    def parse(self, statement):
        return self._parser.parse(statement)
        
    def set_variables(self, variables=dict(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, self.get_universe())
        self._variables = variables
        
    def get_variables(self, *variables):
        if len(variables) == 0:
            return self._variables
        else:
            selected_variables = dict()
            for variable in variables:
                selected_variables[variable] = self._variables[variable]
            return selected_variables
    
    def add_variable(self, variable, value, check=False):
        if check == True:
            self.is_valid(value, self.get_universe())
        variables = self.get_variables()
        if variable in variables.keys():
            raise Exception(f"Variable - '{variable}' is already defined!")
        variables[variable] = value
        self.set_variables(variables)
        
    def set_variable(self, variable, new_value, check=False):
        if check == True:
            self.is_valid(new_value, self.get_universe())
        variables = self.get_variables()
        if variable not in variables.keys():
            raise Exception(f"Variable - '{variable}' has not been defined yet!")
        variables[variable] = new_value
        self.set_variables(variables)
        
    def set_universe(self, universe=FiniteSet(), check=False):
        if check == True:
            for relation in self.get_variables().values():
                if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
                    raise Exception(f"Universe - {universe} not valid for relation - {relation}!") 
        self._universe = FiniteSet(*universe)
        
    def get_universe(self):
        return self._universe
        
    def constant(self,data):
        try:
            variables = self.get_variables()
            return variables[data]
        except:
            raise Exception(f"Variable - '{data}' doesn't exist!")

    def interpret(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return not self.interpret(node.children[0])
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return self.interpret(left) == self.interpret(right)
                elif node.data == 'not_equal':
                    return self.interpret(left) != self.interpret(right)
                elif node.data == 'greater_than':
                    return self.interpret(left) > self.interpret(right)
                elif node.data == 'lower_than':
                    return self.interpret(left) < self.interpret(right)
                elif node.data == 'greater_equal':
                    return self.interpret(left) >= self.interpret(right)
                elif node.data == 'lower_equal':
                    return self.interpret(left) <= self.interpret(right)
                elif node.data == 'conjunction':
                    return self.interpret(left) and self.interpret(right)
                elif node.data == 'disjunction':
                    return self.interpret(left) or self.interpret(right)
                elif node.data == 'implication':
                    return (not self.interpret(left)) or self.interpret(right)
                elif node.data == 'intersection':
                    return self.interpret(left) & self.interpret(right)
                elif node.data == 'union':
                    return self.interpret(left) | self.interpret(right)
                elif node.data == 'difference':
                    return self.interpret(left) - self.interpret(right)
                elif node.data == 'symmetric_difference':
                    return self.interpret(left) ^ self.interpret(right)
                elif node.data == 'multiplication':
                    return self.interpret(left) * self.interpret(right)
                elif node.data == 'composition':
                    return self.interpret(left).composition(self.interpret(right))
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return self.interpret(left).is_lattice() if node.data == 'is' else not self.interpret(left).is_lattice()
                    elif right.data == 'total_order':
                        return self.interpret(left).is_total_order() if node.data == 'is' else not self.interpret(left).is_total_order()
                    elif right.data == 'reflexive':
                        return self.interpret(left).is_reflexive() if node.data == 'is' else not self.interpret(left).is_reflexive()
                    elif right.data == 'symmetric':
                        return self.interpret(left).is_symmetric() if node.data == 'is' else not self.interpret(left).is_symmetric()
                    elif right.data == 'asymmetric':
                        return self.interpret(left).is_asymmetric() if node.data == 'is' else not self.interpret(left).is_asymmetric()
                    elif right.data == 'antisymmetric':
                        return self.interpret(left).is_antisymmetric() if node.data == 'is' else not self.interpret(left).is_antisymmetric()
                    elif right.data == 'transitive':
                        return self.interpret(left).is_transitive() if node.data == 'is' else not self.interpret(left).is_transitive()
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                raise Exception(f"ERROR! Unexpected input!")
        elif isinstance(node, Token):
            node = self.constant(node.value)
            return node
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")

In [16]:
import unittest

class TestNotebook_interpreter(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
    relation2 = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
    relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(4,4)},{1,2,3,4}, True)
    universe1 = {1,2,3}
    
    def test_valid_interpreter(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertEqual(interpreter.get_variables(), variables)
        self.assertEqual(interpreter.get_universe(), self.universe1)
        
    def test_set_universe_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {1,2,3,4,'h','ch'}
        interpreter.set_universe(new_universe, True)
        self.assertEqual(interpreter.get_universe(), new_universe)
        
    def test_set_universe_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {2,3,4,'h','ch'}
        self.assertRaises(Exception, interpreter.set_universe,new_universe, True)
        
    def test_set_variables_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation2}
        interpreter.set_variables(new_variables, True)
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variables_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation3}
        self.assertRaises(Exception, interpreter.set_variables,new_variables, True)
        
    def test_set_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.set_variable('S',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'T',self.relation2, True)
        
    def test_set_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'R',self.relation3, True)
        
    def test_add_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.add_variable('T',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1,'T':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_add_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'R',self.relation2, True)
        
    def test_add_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'T',self.relation3, True)
        
    def test_interpret_formula_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive and R | S is not transitive")
        self.assertEqual(interpreter.interpret(tree), False)
        
    def test_interpret_formula_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_interpret_formula_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)       
        U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
        V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
        interpreter.add_variable('U',U, True)
        interpreter.add_variable('V',V, True)       
        tree = interpreter.parse("V is lattice and U|V is transitive ->  V is total_order")
        self.assertEqual(interpreter.interpret(tree), True)
    
        
unittest.main(argv=[''], verbosity=1, exit=False)

........................................................................................................
----------------------------------------------------------------------
Ran 104 tests in 1.687s

OK


In [17]:
relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
relation2 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

interpreter = Interpreter({'R':relation1,'S':relation2}, {1,2,3}, True)
# #interpreter.set_variables({'R':relation1,'X':HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3,4}, True)}, True) #Error

In [18]:
tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpreter.interpret(tree)}]")

[TREE]
implication
  conjunction
    is
      R
      transitive
    is
      S
      transitive
  is
    union
      R
      S
    transitive
[/TREE]

[RESULT -> True]


In [19]:
# tree = interpreter.parser.parse("R is transitive and S is transitive and not R | S is transitive")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [20]:
# U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
# V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

# interpreter.add_variable('U',U, True)
# interpreter.add_variable('V',V, True)
# tree = interpreter.parser.parse("V is lattice and U|V is transitive ->  V is total_order")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables()}")

In [21]:
# V = PartialOrder({(1,1),(2,2),(3,3)}, {1,2,3},True)

# #interpreter.add_variable('V',V) #Error - already defined variable
# interpreter.set_variable('V',V,True)
# #interpreter.set_universe({1,2},True) #Error - not valid universe

# tree = interpreter.parser.parse("V != V ->  S is total_order")




# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables('V','S')}")

In [22]:
#interpreter.get_variables()

In [23]:
#interpreter.get_universe()